# Workshop 3: IIR and FIR filter Design

## Introduction

This workshop explores some key concepts in digital filter design. It assumes that you are already familiar with the main characteristics of filters (low-pass, high-pass, band-pass, band-stop, order, cut-off frequency, Bode plots, magnitude, phase, ripple, roll-off…….etc.) and that you can already compare some alternative analog filter designs (e.g. Butterworth, Chebychev, Bessel). If you are unsure of these terms then do research them before continuing.

The workshop builds on this knowledge to look at key differences between the two important categories of filter that can be implemented with DSP techniques: FIR and IIR. So by the end of this lab you will be able to:

* Explain the meaning of “FIR filter” and “IIR filter”.
* Critically evaluate their relative advantages and disadvantages with respect to some typical applications.

In this notebook we are going to look at how to use [`numpy`](https://numpy.org) and [`scipy`](https://scipy.org/) which are Python packages that add mathmetical and scientific tools to Python. [`numpy`](https://numpy.org/doc/stable/user/numpy-for-matlab-users.html) has a specific guide in using it for Matlab users 


## Filter design tools

For a very wide range of applications nowadays, engineers can take advantage of user-friendly, often free, software tools which quickly produce an optimum filter design to meet their specific requirements. `scipy` provides [filter design](https://docs.scipy.org/doc/scipy/tutorial/signal.html#filter-design) tools similar to those found in Matlab which are freeware and therefore increasingly popular in use.

Looking at the `scipy` [Signal processing](https://docs.scipy.org/doc/scipy/reference/signal.html#filter-design) reference API find out what the following commands do:

* `butter` and `buttord`
* `ellip` and `ellipord`
* `firwin` and `kaiserord`

You can also look at the general filter design documentation which gives examples of some of these functions in use.

### Answers

* `butter`
* `butterord`
* `ellip`
* `ellipord`
* `firwin`
* `kaiserord`


## Digital Butterworth filter

A general IIR filter is given by transfer function of: 

$$
 H(z)=\dfrac{\sum_0^M b_i z^{-i}}{1+\sum_0^N a_j z^{-j}}
$$

Use the `butter` command to generate a 2nd order low-pass Butterworth filter with a cut-off frequency of 40Hz and a sampling requency of 500Hz - in the code below you will need to assign the correct values to N and Wn  before you .

In [ ]:
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np

# initialise values
Fs = 500
N = 2
Wn = 40

# get values for coefficients
b, a = signal.butter(N, Wn, "low", fs=Fs)

# get system for designed filter
sys = signal.dlti(b, a)
# Calculate impulse response
t, y = signal.dimpulse(sys, n=1500)
hb = np.squeeze(y)
t = t * 1 / Fs
fig, ax2 = plt.subplots(tight_layout=True)
ax2.plot(t, hb, "b", alpha=0.75)
ax2.grid()
ax2.set_title(
    f"Impulse Response of order 2 Butterworth IIR Filter"
    + f"($F_c={Wn}$ Hz)"
)
ax2.set_ylabel("Amplitude(dB)", color="C0")
ax2.set(xlabel="time (s)", xlim=(0, 1))

# Calculate frequency response of filter
w, h = signal.freqz(b, a, worN=1024, fs=Fs)
# plot frequency response
fig, ax1 = plt.subplots(tight_layout=True)
ax1.grid()
ax1.set_title(
    f"Frequency Response of order 2 Butterworth IIR Filter"
    + f"($F_c={Wn}$ Hz)"
)
ax1.axvline(50, color="black", linestyle=":", linewidth=0.8)
ax1.plot(w, 20 * np.log10(abs(h)), "C0")
ax1.set_ylabel("Amplitude(dB)", color="C0")
ax1.set(xlabel="Frequency (Hz)", xlim=(0, Fs / 2))

Check that the features of the filter agree with what you expect. 

Is this an FIR or IIR filter? 

How do you know? 

## Filtering by Convolution in the Time Domain

The time domain signal above shows contains a physiological signal buried in high frequency noise as can be seen from the signal spectrum. Therefore we need to filter the signal using a lowpass filter to remove the high frequency noise. 

To load the signal into MATLAB make sure you have downloaded the medical signal.dat file from My Beckett and that your current working directory on MATLAB is wherever that signal is. Then use the code below (adding to previous code): 

In [ ]:
xmed = np.loadtxt("data/medical_signal.dat")
T = 1 / Fs
L = len(xmed)
print(L)
t = np.linspace(0, ((L - 1) * T), L)  # Time vector

fig, ax1 = plt.subplots(tight_layout=True)
ax1.plot(1000 * t, xmed)
ax1.grid()
ax1.set_title("Medical Signal")
ax1.set_ylabel("xmed", color="C0")
ax1.set(xlabel="time (ms)", xlim=(0, 3000))

In [ ]:
# from pytictoc import TicToc
# timer = TicToc()

# timer.tic()
y4 = signal.convolve(xmed, hb)
# perform convolution with your Butterworth filter
# timer.toc()
fig, ax1 = plt.subplots(tight_layout=True)
ax1.plot(
    1000 * t, np.real(y4[1:1501])
)  # display the filtered waveform
ax1.grid()
ax1.set_title("Time Domain Filtered Medical Signal")
ax1.set_ylabel("xmed filtered", color="C0")
ax1.set(xlabel="time (ms)", xlim=(0, 3000))

Has the filter worked as expected? 

## Filtering by Multiplication in the Frequency Domain 

An alternative method to filter the signal is to transform it into the frequency domain, multiply by the filter frequency response and then transform back into the time domain. This might sound like a very roundabout way of doing things, but it is actually very practicable, as we shall see. You can use the code below as a guide to what is needed (this needs to ensure the medical signal data is loaded as xmed). 

In [ ]:
from scipy import fft

X5 = fft.fft(xmed) / len(xmed)
f = Fs / len(X5) * np.linspace(0, (len(X5) - 1), len(X5))

fig, ax1 = plt.subplots(tight_layout=True)
ax1.plot(f, np.abs(X5))  # display the filtered waveform
ax1.grid()
ax1.set_title("FFT of Mystery Signal")

In [ ]:
HBC = fft.fft(hb) / len(hb)
HBR = HBC.transpose()
f = Fs / len(HBR) * np.linspace(0, (len(HBR) - 1), len(HBR))

fig, ax1 = plt.subplots(tight_layout=True)
ax1.plot(f, np.abs(HBR))  # display the filtered waveform
ax1.grid()
ax1.set_title("FFT of Filter Impulse Repsonse")

In [ ]:
Y5 = X5 * HBR

fig, ax1 = plt.subplots(tight_layout=True)
ax1.plot(f, np.real(Y5))  # display the filtered waveform
ax1.grid()
ax1.set_title("Frequency domain multiplication")

In [ ]:
y5 = fft.ifft(Y5) * (len(Y5) * len(Y5))

fig, ax1 = plt.subplots(tight_layout=True)
ax1.plot(
    1000 * t, np.real(y5)
)  # display the filtered waveform
ax1.grid()
ax1.set_title("Frequency Domain Filtered Medical Signal")
ax1.set_ylabel("xmed", color="C0")
ax1.set(xlabel="time (ms)", xlim=(0, 3000))

Compare the results of filtering in time and frequency domains. The should confirm that:

<img src="images/workshop3figs-1.png" width=600 alt="Convolution in time domain is equivalent to multiplication in frequency domain">

## Further Work

The next two sections are further work sections that will enable you to gian mroe than just a threshold pass for this workshop.

## FIR filter comparison

Repeat the work done above using the Butterworth filter for an equivalent FIR filter. What differences do you notice?

In [ ]:
# code for FIR filter
from scipy import signal, fft
import matplotlib.pyplot as plt
import numpy as np

# initialize values
Fs = 500
N = 2
Wn = 40

What are the differences?

## Comparing processing speed for real-time filtering

Looking at the documentation for [pytictoc](https://pypi.org/project/pytictoc/) Python module work out how to time the speed of esecution of the filters in the sections above in the context of this medical signal processing example and critically evaluate the results. Remember that you should only consider the sections of code that would be involved in real-time processing of the physiological signal --- so can assume impusle response and FFT of the filter are already known - it is just the input signal that needs to be processed using the filter. You should also consider combining the sections of code together and do the plots at end for better timing.

